In [2]:
from durable.lang import *

with ruleset('attributes'):
    @when_all(pri(3), m.amount < 300)
    def first_detect(c):
        print('attributes P3 ->{0}'.format(c.m.amount))
        
    @when_all(pri(2), m.amount < 200)
    def second_detect(c):
        print('attributes P2 ->{0}'.format(c.m.amount))
        
    @when_all(pri(1), m.amount < 100)
    def third_detect(c):
        print('attributes P1 ->{0}'.format(c.m.amount))
                
assert_fact('attributes', { 'amount': 50 })
assert_fact('attributes', { 'amount': 150 })
assert_fact('attributes', { 'amount': 250 })

attributes P1 ->50
attributes P2 ->50
attributes P3 ->50
attributes P2 ->150
attributes P3 ->150
attributes P3 ->250


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [3]:
from durable.lang import *

with ruleset('expense'):
    # this rule will trigger as soon as three events match the condition
    @when_all(count(3), m.amount < 100)
    def approve(c):
        print('approved {0}'.format(c.m))

    # this rule will be triggered when 'expense' is asserted batching at most two results       
    @when_all(cap(2),
              c.expense << m.amount >= 100,
              c.approval << m.review == True)
    def reject(c):
        print('rejected {0}'.format(c.m))

post_batch('expense', [{ 'amount': 10 },
                                    { 'amount': 20 },
                                    { 'amount': 100 },
                                    { 'amount': 30 },
                                    { 'amount': 200 },
                                    { 'amount': 400 }])
assert_fact('expense', { 'review': True })

approved [{'amount': 30}, {'amount': 20}, {'amount': 10}]
rejected [{'expense': {'amount': 400}, 'approval': {'review': True}}, {'expense': {'amount': 200}, 'approval': {'review': True}}]
rejected [{'expense': {'amount': 100}, 'approval': {'review': True}}]


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [4]:
from durable.lang import *

with statechart('expense2'):
    # initial state 'input' with two triggers
    with state('input'):
        # trigger to move to 'denied' given a condition
        @to('denied')
        @when_all((m.subject == 'approve') & (m.amount > 1000))
        # action executed before state change
        def denied(c):
            print ('denied amount {0}'.format(c.m.amount))
        
        @to('pending')    
        @when_all((m.subject == 'approve') & (m.amount <= 1000))
        def request(c):
            print ('requesting approve amount {0}'.format(c.m.amount))
    
    # intermediate state 'pending' with two triggers
    with state('pending'):
        @to('approved')
        @when_all(m.subject == 'approved')
        def approved(c):
            print ('expense approved')
            
        @to('denied')
        @when_all(m.subject == 'denied')
        def denied(c):
            print ('expense denied')
    
    # 'denied' and 'approved' are final states    
    state('denied')
    state('approved')
        
# events directed to default statechart instance
post('expense2', { 'subject': 'approve', 'amount': 100 })
post('expense2', { 'subject': 'approved' })

# events directed to statechart instance with id '1'
post('expense2', { 'sid': 1, 'subject': 'approve', 'amount': 100 })
post('expense2', { 'sid': 1, 'subject': 'denied' })

# events directed to statechart instance with id '2'
post('expense2', { 'sid': 2, 'subject': 'approve', 'amount': 10000 })

requesting approve amount 100
expense approved
requesting approve amount 100
expense denied
denied amount 10000


{'sid': '2', 'id': 'sid-2', '$s': 1, 'running': True}

In [5]:
from durable.lang import *

with flowchart('expense3'):
    # initial stage 'input' has two conditions
    with stage('input'): 
        to('request').when_all((m.subject == 'approve') & (m.amount <= 1000))
        to('deny').when_all((m.subject == 'approve') & (m.amount > 1000))
    
    # intermediate stage 'request' has an action and three conditions
    with stage('request'):
        @run
        def request(c):
            print('requesting approve')
            
        to('approve').when_all(m.subject == 'approved')
        to('deny').when_all(m.subject == 'denied')
        # reflexive condition: if met, returns to the same stage
        to('request').when_all(m.subject == 'retry')
    
    with stage('approve'):
        @run 
        def approved(c):
            print('expense approved')

    with stage('deny'):
        @run
        def denied(c):
            print('expense denied')

# events for the default flowchart instance, approved after retry
post('expense3', { 'subject': 'approve', 'amount': 100 })
post('expense3', { 'subject': 'retry' })
post('expense3', { 'subject': 'approved' })

# events for the flowchart instance '1', denied after first try
post('expense3', { 'sid': 1, 'subject': 'approve', 'amount': 100})
post('expense3', { 'sid': 1, 'subject': 'denied'})

# event for the flowchart instance '2' immediately denied
post('expense3', { 'sid': 2, 'subject': 'approve', 'amount': 10000})

requesting approve
requesting approve
expense approved
requesting approve
expense denied
expense denied


{'sid': '2', 'id': 'sid-2', '$s': 1, 'running': True}

In [6]:
from durable.lang import *

with statechart('expense10'):
    # initial state 'input' with two triggers
    with state('input'):
        # trigger to move to 'denied' given a condition
        @to('denied')
        @when_all((m.subject == 'approve') & (m.amount > 1000))
        # action executed before state change
        def denied(c):
            print ('denied amount {0}'.format(c.m.amount))
        
        # @to('pending')    
        # @when_all((m.subject == 'approve') & (m.amount <= 1000))
        # def request(c):
        #     print ('requesting approve amount {0}'.format(c.m.amount))

        @to('approved')
        @when_all(+m.amount)
        def all(c):
            print(c)
    
    # intermediate state 'pending' with two triggers
    with state('pending'):
        @to('approved')
        @when_all(m.subject == 'approved')
        def approved(c):
            print ('expense approved')
            
        @to('denied')
        @when_all(m.subject == 'denied')
        def denied(c):
            print ('expense denied')
    
    # 'denied' and 'approved' are final states    
    state('denied')
    state('approved')
        
# events directed to default statechart instance
post('expense10', { 'subject': 'approve', 'amount': 100 })
# post('expense7', { 'subject': 'approved' })

# # events directed to statechart instance with id '1'
# post('expense2', { 'sid': 1, 'subject': 'approve', 'amount': 100 })
# post('expense2', { 'sid': 1, 'subject': 'denied' })

# # events directed to statechart instance with id '2'
# post('expense2', { 'sid': 2, 'subject': 'approve', 'amount': 10000 })

Exception: Chart expense10 has more than one start state pending

In [ ]:
from durable.lang import *

with ruleset('expense14'):
    # this rule will trigger as soon as three events match the condition
    @when_all(count(3), m.amount < 100)
    def approve(c):
        print('approved {0}'.format(c.m))

    # this rule will be triggered when 'expense' is asserted batching at most two results       
    @when_all(cap(2),
              c.expense << m.amount >= 300,
              c.approval << m.review == True)
    def reject(c):
        print('rejected {0}'.format(c.m))

    @when_all(cap(2),
              c.expense << m.amount >= 100,
              c.approval << m.review == True)
    def reject2(c):
        print('rejected2 {0}'.format(c.m))

post_batch('expense14', [{ 'amount': 10 },
                                    { 'amount': 20 },
                                    { 'amount': 100 },
                                    { 'amount': 30 },
                                    { 'amount': 200 },
                                    { 'amount': 400 }])
# assert_fact('expense13', { 'review': True })

approved [{'amount': 30}, {'amount': 20}, {'amount': 10}]


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [ ]:
from durable.lang import *

with ruleset('bookstore'):
    # this rule will trigger for events with status
    @when_all(+m.status)
    def event(c):
        print('bookstore-> Reference {0} status {1}'.format(c.m.reference, c.m.status))

    @when_all(+m.name)
    def fact(c):
        print('bookstore-> Added {0}'.format(c.m.name))
        
    # this rule will be triggered when the fact is retracted
    @when_all(none(+m.name))
    def empty(c):
        print('bookstore-> No books')

# will not throw because the fact assert was successful 
assert_fact('bookstore', {
    'name': 'The new book',
    'seller': 'bookstore',
    'reference': '75323',
    'price': 500
})

# will throw MessageObservedError because the fact has already been asserted 
try:
    assert_fact('bookstore', {
        'reference': '75323',
        'name': 'The new book',
        'price': 500,
        'seller': 'bookstore'
    })
except BaseException as e:
    print('bookstore expected {0}'.format(e.message))

# will not throw because a new event is being posted
post('bookstore', {
    'reference': '75323',
    'status': 'Active'
})

# will not throw because a new event is being posted
post('bookstore', {
    'reference': '75323',
    'status': 'Active'
})

retract_fact('bookstore', {
    'reference': '75323',
    'name': 'The new book',
    'price': 500,
    'seller': 'bookstore'
})

bookstore-> Added The new book
bookstore expected Message has already been observed: {"reference": "75323", "name": "The new book", "price": 500, "seller": "bookstore"}
bookstore-> Reference 75323 status Active
bookstore-> Reference 75323 status Active
bookstore-> No books


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [7]:
nodes = [
    {
        'bin': '0-33',
        'depth': 0,
        'score': 47.1,
        'type': 'read'
        # 'size': 50,
        # 'time': 13.8
    },
    # {
    #     'bin': 'read-0-16',
    #     'depth': 1,
    #     'score': 40.1,
    #     # 'size': 50,
    #     # 'time': 13.8
    # },
    # {
    #     'bin': 'read-16-33',
    #     'depth': 1,
    #     'score': 45.7,
    #     # 'size': 50,
    #     # 'time': 13.8
    # },
    {
        'bin': '0-33',
        'depth': 0,
        'score': 47.1,
        'type': 'write'
        # 'size': 50,
        # 'time': 13.8
    },
    {
        'bin': '0-33',
        'depth': 0,
        'score': 67.5,
        'type': 'metadata'
        # 'size': 50,
        # 'time': 13.8
    },
    {
        'bin': '0-33',
        'depth': 0,
        'score': 37.5,
        'type': 'metadata2'
        # 'size': 50,
        # 'time': 13.8
    },
    {
        'bin': '0-33',
        'depth': 0,
        'score': 77.5,
        'type': 'metadata3'
        # 'size': 50,
        # 'time': 13.8
    },
    {
        'type': 'ready'
    }
]




In [34]:
with statechart('analysis45'):

    with state('start'):
        @when_all(pri(0))
        def keep_nodes(c):
            print('keep_nodes')
            print(c.m)
            if not c.s.nodes:
                c.s.nodes = []
            c.s.nodes.append(c.m)

        @to('analyze_root')
        @when_all(m.type == 'ready')
        def analyzing_root(c):
            print('-------')
            print('analyzing_roots')
            c.s.max_root_score = 0.0
            print(c.root)

        @to('analyze_node')
        @when_all(m.depth == 1)
        def analyzing_node(c):
            print('-------')
            print('analyzing_nodes')

    with state('analyze_root'):

        @to('analyze_root')
        # @when_all(c.r1 << (m.depth == 0), 
        #           c.r2 << (m.depth == 0) & (m.type != c.r1.type) & (m.score > c.r1.score))
        @when_all(m.depth == 0)
        def analyze_root(c):
            print('-------')
            print('analyze_root')
            print(c.s.max_root_score)
            c.s.max_root_score = c.m.score
            # print(c.s.nodes)

        @to('analyze_node')
        @when_all(pri(1))
        def analyzed_root(c):
            print('analyzed_roots')
            c.s.max_root_score = c.m.score

    with state('analyze_node'):

        @when_all(m.depth == 1)
        def analyze_node(c):
            print('-------')
            print('analyze_node')
            print(c.m)
            # print(c.s.nodes)

    # state('end')


In [35]:
for node in nodes:
    # print(node)
    post('analysis45', node)

-------
analyzing_roots
None
-------
analyze_root
0.0
-------
analyze_root
77.5
-------
analyze_root
37.5
-------
analyze_root
67.5
-------
analyze_root
47.1
analyzed_roots
